In [7]:
import re
import json
import string

import nltk
from ClassifierBasedGermanTagger import ClassifierBasedGermanTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation
from sklearn.externals import joblib


In [2]:
filename = 'german_tagger.sav'

In [3]:
loaded_model = joblib.load(filename)

In [24]:
def pos_tag_sentence(model, sent):
    return model.tag(sent.split())

In [25]:
def clean_string(string, regex=r'[^a-zA-Z\s0-9ÄÖÜäöü.?,;]+'):
    return re.sub(regex, '', string)

## read json data

In [26]:
with open('dienstleistungen.json') as data_file:    
    data = json.load(data_file)

In [27]:
def parse_names(data):
    names = []
    for entry in data['data']:
        if entry['name'] == tuple():
            tmp = clean_string(entry['name'][0])
            names.append(tmp)
        else:
            tmp = clean_string(entry['name'])
            names.append(tmp)
    return names

In [28]:
names = parse_names(data)

## clustering

In [45]:
punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.snowball.GermanStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def get_clusters(sentences):
    tf_idf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
    affinity_propagation = AffinityPropagation(affinity="precomputed", damping=0.5)
    affinity_propagation.fit(similarity_matrix)

    labels = affinity_propagation.labels_
    cluster_centers = affinity_propagation.cluster_centers_indices_

    tagged_sentences = zip(sentences, labels)
    clusters = {}

    for sentence, cluster_id in tagged_sentences:
        clusters.setdefault(sentences[cluster_centers[cluster_id]], []).append(sentence)

    return clusters

def print_clusters(clusters):
    for cluster in clusters:
        print(cluster, ':')
        for element in clusters[cluster]:
            print('  - ', element)

## text clustering

In [49]:
print_clusters(get_clusters(names))

Melderegisterauskunft online für Einzelabfrager :
  -  Melderegisterauskunft sperren
  -  Melderegisterauskunft einholen
  -  Melderegisterauskunft online für Einzelabfrager
  -  Melderegisterauskunft online für registrierte Grokunden
  -  Melderegisterauskunft  Auskunft für Eigentümer bzw. Wohnungsgeber und Vermieter
Umsatzsteuer  Voranmeldung  Allgemeines :
  -  Niederlassungserlaubnis allgemein
  -  Umsatzsteuer  Voranmeldungen, Jahreserklärungen
  -  Kinderfreibeträge  Allgemeine Informationen
  -  Umsatzsteuer  Kleinunternehmer
  -  Umsatzsteuer  Voranmeldung  Dauerfristverlängerung
  -  Umsatzsteuer  Voranmeldung  Allgemeines
Beglaubigung von Urkunden für das Ausland ApostilleLegalisation :
  -  Beglaubigung von Kopien
  -  Beglaubigung von Unterschriften
  -  Beglaubigung von Fingerabdrücken zur Vorlage bei ausländischen Behörden
  -  Beglaubigung von Urkunden für das Ausland ApostilleLegalisation
  -  Vereinigungs und Teilungsanträge Grundstücke, Beglaubigung
  -  Beglaubigung 

## test clustering with POS

In [46]:
def pos_tag_names(model, data, pos=['NN', 'NE']):
    tmp = []
    for sent in data:
        pos_tags = pos_tag_sentence(model, sent)
        for tags in pos_tags:
            if tags[1] in pos:
                tmp.append(tags[0])
    return tmp

In [47]:
test = pos_tag_names(loaded_model, names)

In [48]:
print_clusters(get_clusters(test))

Arbeitsleben :
  -  Kartenverkauf
  -  Grunderwerbsteuer
  -  Erlaubnis
  -  BeratungBetreuung
  -  Arbeitsleben
  -  Arbeitsleben
Fahrerkarte :
  -  Fahrerkarte
  -  StadtplanerListe
Wohnberechtigungsschein :
  -  Wohnberechtigungsschein
  -  Pflanzenschutz
  -  Herausgabevollstreckung
Erbrecht :
  -  Erbrecht
  -  Erbrecht
  -  Erbrecht
Erbverträgen :
  -  NichtEUMitgliedstaat
  -  Eltern
  -  Erbverträgen
  -  Erbverträgen
Gehwegen :
  -  Gehwegen
  -  Gehwegen
  -  Gehwegen
Geburtsurkunde :
  -  Geburtsurkunde
  -  Geburtsurkunde
  -  Unschädlichkeitszeugnis
  -  Geburtsurkunde
  -  Eigenantrag
Beglaubigung :
  -  Beglaubigung
  -  Beglaubigung
  -  Aufenthaltserlaubnis
  -  Beglaubigung
  -  Beglaubigung
  -  Beglaubigung
  -  Beglaubigung
  -  Beglaubigung
  -  Beglaubigung
  -  Beglaubigungen
Schuldnerverzeichnis :
  -  Hörstörung
  -  Schuldnerverzeichnis
  -  Schuldnerverzeichnis
  -  Schuldnerverzeichnis
  -  Schuldnerverzeichnis
  -  Schuldnerverzeichnis
  -  Schuldnerverzei